## Abstract 

Title: Predicting Scope 3 Emissions using Machine Learning: A Novel Approach

The following research of mine is based on a study conducted by Serafeim, George and Velez Caicedo, Gladys. 2022. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022.

I would like to thank the authors for sharing their methodology and data, which allows me to independently conduct research and modeling and then compare the results with the conclusions of the researchers thanks to which I have a great opportunity to learn and to lead the research in new directions by updating the model.

## Data Guidence

Copyright © 2022 by George Serafeim and Gladys Velez Caicedo. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022		
Funding for this research was provided in part by Harvard Business School.		
		
Data source: 		
“Serafeim, George and Velez Caicedo, Gladys. 2022. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022		
		
GUIDANCE:		
Column A "Year" is the year in which the environmental impact was incurred by the firm's operations.		
		
Column B "Company Name" is the name of the issuer.		
		
Column C "Country" is the country in which the companies' headquarters are located.		
		
Column D "Industry" refers to the Exiobase industry category to which the firm belongs: We provide only the Exiobase industries here as they are open source, but in our paper, we use GICS taxonomy as fixed effects.		
All Exiobase industries are based on the International Standard Industrial Classification Revision 3.1 (ISIC). 	To learn more about ISIC and a comprehensive list of industries included, please refer to: unstats.un.org/unsd/statcom/doc02/isic.pdf	
	For example, the term "nec" refers to "not elsewhere classified."	
		
Column E "GHG Intensity (Sales)" is the monetized GHG impact of the firm's operations during the specific year indicated in column A divided by revenue in that year		
		
Column F "GHG Intensity (Op Inc)" is the monetized GHG impact of the firm's operations during the specific year indicated in column A divided by operating income in that year		
		
Column G "Total GHG Environmental Cost (Scope 1, 2, 3) " is the total monetized GHG environmental impact of Scope 1, 2, and 3 emissions of the firm's operations during the specific year indicated in Column A.		
		
Columns H-J are Scope 1, 2, 3 Emissions		
Each scope of emissions is defined by the GHG Protocol. More information can be found at the Greenhouse Gas Protocol: https://ghgprotocol.org/		
	Column H:	 Scope 1 Emissions: emissions from direct operations that occur from sources that are controlled or owned by the firm 
	Column I:	 Scope 2 Emissions: emissions associated with the purchase of electricity, steam, heat, or cooling as a result of the firm's energy use 
	Column J:	 Scope 3 Emissions: emissions from 15 categories that are result of activities from assets not owned or controlled by the reporting firm, not within a firm's scope 1 and 2 boundary and occur through the value chain. 
		
Columns K-BC are fiveteen Scope 3 emissions category types in alphabetical order followed by an indicator variable denoting if the data point is company reported (0) or if the data point is predicted via machine learning (1)		
	Column K	Business Travel
	Column N:	Capital Goods
	Column Q:	Downstream Leased Assets
	Column T:	Downstream Transportation and Distribution
	Column W:	Employee Commuting
	Column Z:	End of Life Treatment of Sold Products
	Column AC:	Franchises
	Column AF:	Fuel-and-energy-related activities (not included in Scope 1 or 2)
	Column AI:	Investments
	Column AL:	Processing of Sold Products
	Column AO:	Purchased Goods and Services
	Column AR:	Upstream Leased Assets
	Column AU:	Upstream Transportation and Distribution
	Column AX:	Use of Sold Products
	Column BA:	Waste Generated in Operations
		
The dataset is a combination of primary firm reported emissions data supplemented with Scope 3 predictions by category.		
Our methodology takes firm reported values first and incorporates imputations only when companies' self-reported emissions data are not publicly available.		
If the data point is imputed, the Scope 3 category "Imputed" value is 1.		
If the data point is company reported, the Scope 3 category "Imputed" value is 0.		
The Scope 3 category "Test" column indicates if the data point was used to "train" or "test" the machine learning model. If no company value is reported, the value. Is set to "none". 		
		
Other Notes:		
The "Final Raw Sample(0%)" tab includes all raw outputs, discounted at 0%, from our environmental impact calculation methodology. The Social Cost of Carbon discounted at 0% applied here is roughly $300 USD per metric ton of emissions.		
The "Final Raw Sample(3%)" tab includes all raw outputs, discounted at 3%, from our environmental impact calculation methodology. The Social Cost of Carbon discounted at 3% applied here is roughly $100 USD per metric ton of emissions.		
All observations in the tabs are sorted by 1) Year in descending order, 2) Industry in alphabetical order, and 3) Environmental Intensity (Sales) in descending order.		
		
		
Also, if you are a researcher planning to use the data in an academic research project, please email us and we will send you a file including ISINs to facilitate merging with other datasets.		
Our team can be reached at: ImpactWeightedAccounts@hbs.edu

In [63]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

## Load data from Kedro catalog 

In [55]:
# catalog.list()

In [39]:
# This method does not work for MakinaRocks pipeline inside the notebook
# df = catalog.load("scope3_data_3")
# features = catalog.load("params:feature_options")

[11/03/23 14:29:05] INFO     Loading data from 'scope3_data_3' (ExcelDataSet)...                ]8;id=315866;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=617937;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

[11/03/23 14:29:10] INFO     Loading data from 'params:feature_options' (MemoryDataset)...      ]8;id=704093;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=868934;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

## Load data traditional way for MakinaRocks pipeline

In [73]:
# Define the path to the Excel file
file_path = '../data/01_raw/IWA-External-Scope-3-Data.xlsx'

# Read the Excel file, specifying the sheet name
df = pd.read_excel(file_path, sheet_name='3%')

import yaml

# Define the path to the YAML file
yaml_file_path = '../conf/base/parameters/data_processing.yml'

# Load the YAML file
with open(yaml_file_path, 'r') as file:
    data_processing_config = yaml.safe_load(file)

# Extract the features list
features_list = data_processing_config['feature_options']['features']

# Construct the desired dictionary format
features = {'features': features_list}

In [74]:
features


{
    'features': [
        'Industry (Exiobase)',
        'Business Travel',
        'Capital Goods',
        'Downstream Leased Assets',
        'Downstream Transportation and Distribution',
        'Employee Commuting',
        'End of Life Treatment of Sold Products',
        'Fuel-and-energy-related activities (not included in Scope 1 or 2)',
        'Processing of Sold Products',
        'Purchased Goods and Services',
        'Upstream Leased Assets',
        'Upstream Transportation and Distribution',
        'Use of Sold Products',
        'Waste Generated in Operations',
        'Scope 3'
    ]
}

## Preprocessing

## Remove missing values

In [58]:
# def _remove_missing_values(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Function to remove all rows with missing values in a pandas dataframe.

#     Args:
#         df (pd.DataFrame): Input pandas DataFrame

#     Returns:
#         pd.DataFrame: Output DataFrame with rows containing missing values removed.
#     """

#     df_cleaned = df.dropna()

#     return df_cleaned

In [66]:
from typing import List, Union

def _remove_rows_with_missing_values(df: pd.DataFrame, columns: Union[str, List[str]] = None) -> pd.DataFrame:
    """
    Remove all rows containing missing values either from the whole DataFrame or from specific columns.

    Args:
        df (pd.DataFrame): Input DataFrame.
        columns (Union[str, List[str]], optional): Column or list of columns to consider for row removal. 
                                                   If None, consider all columns. Default is None.
    
    Returns:
        pd.DataFrame: DataFrame with rows containing missing values removed.
    """
    
    if columns is not None:
        return df.dropna(subset=columns)
    else:
        return df.dropna()

## Outliers detection

In [12]:
# def _outlier_removal(df: pd.DataFrame) -> pd.DataFrame:
#     # Identify numerical columns
#     numerical_cols = df.select_dtypes(include=['number']).columns

#     # Initialize the IsolationForest model
#     clf = IsolationForest(contamination=0.2)  # contamination: proportion of outliers in the data set

#     # Fit the model on numerical columns
#     clf.fit(df[numerical_cols])

#     # Get outlier predictions
#     outlier_predictions = clf.predict(df[numerical_cols])

#     # Remove outliers from the original DataFrame based on the predictions
#     df_filtered = df[outlier_predictions == 1]

#     return df_filtered


In [ ]:
from sklearn.ensemble import IsolationForest

def _remove_outliers_isolation_forest(df: pd.DataFrame, contamination: float = 0.2) -> pd.DataFrame:
    """
    Remove outliers using the Isolation Forest algorithm.

    Args:
        df (pd.DataFrame): Input DataFrame with numerical columns.
        contamination (float): Proportion of outliers in the dataset.
    
    Returns:
        pd.DataFrame: DataFrame with outliers removed.
    """

    # Identify numerical columns
    numerical_cols = df.select_dtypes(include=['number']).columns

    # Initialize the IsolationForest model
    clf = IsolationForest(contamination=contamination) 

    # Fit the model on numerical columns
    clf.fit(df[numerical_cols])

    # Get outlier predictions
    outlier_predictions = clf.predict(df[numerical_cols])

    # Remove outliers from the original DataFrame based on the predictions
    df_filtered = df[outlier_predictions == 1]

    return df_filtered

In [59]:
# def preprocess_scope3(scope3_data: pd.DataFrame, parameters: Dict) -> pd.DataFrame:
#     """Preprocesses the Scope 3 data.

#     Args:
#         scope3_data: Raw data.
        
#     Returns:
#         Preprocessed data, with missing values removed.
#     """
    
#     df = scope3_data[parameters["features"]]
#     df = _remove_missing_values(df)
#     preprocessed_data = df
    
#     return preprocessed_data

In [79]:
def preprocess_data(data: pd.DataFrame, parameters: Dict) -> pd.DataFrame:
    """Preprocesses data.

    Args:
        data: Raw data.
        
    Returns:
        Preprocessed data, with missing values removed.
    """
    # Example
    df = data[parameters["features"]]
    df = _remove_rows_with_missing_values(df)
    df = _remove_outliers_isolation_forest(df)
    preprocessed_data = df
    
    return preprocessed_data

In [83]:
preprocessed_data = preprocess_data(df, features)

In [84]:
preprocessed_data

,Industry (Exiobase),Business Travel,Capital Goods,Downstream Leased Assets,Downstream Transportation and Distribution,Employee Commuting,End of Life Treatment of Sold Products,Fuel-and-energy-related activities (not included in Scope 1 or 2),Processing of Sold Products,Purchased Goods and Services,Upstream Leased Assets,Upstream Transportation and Distribution,Use of Sold Products,Waste Generated in Operations,Scope 3
0,"Recreational, cultural and sporting activities...",-2.496564e+05,-4.757744e+04,-3.443104e+04,-2.998176e+04,-7.278893e+04,-1.405697e+05,-1.005045e+04,-3.800732e+06,-4.032700e+04,-115870.418895,-1.622937e+05,-2.517427e+05,-1.680007e+04,-4.972821e+06
1,"Recreational, cultural and sporting activities...",-3.122127e+05,-3.505706e+04,-1.100655e+05,-5.560756e+05,-4.256929e+04,-1.260005e+05,-1.206509e+04,-7.884424e+05,-9.811424e+04,-135220.095921,-3.835789e+05,-4.218799e+06,-8.513858e+04,-6.903339e+06
3,Other land transport,-2.916110e+05,-4.613737e+07,-4.035421e+05,-3.414649e+05,-2.702125e+05,-1.260005e+05,-1.981635e+06,-8.735469e+06,-1.839437e+07,-223065.353185,-9.207954e+05,-5.275860e+04,-2.550743e+05,-8.343143e+07
4,Extraction of crude petroleum and services rel...,-1.641991e+06,-2.500092e+06,-2.649768e+05,-1.754950e+06,-5.172055e+05,-2.835297e+06,-1.627649e+06,-6.753308e+06,-7.203999e+06,-113144.390867,-2.048789e+06,-4.953517e+07,-4.679207e+05,-7.726450e+07
5,Computer and related activities (72),-6.036701e+05,-2.100692e+06,-1.167810e+05,-5.554496e+05,-1.023939e+06,-2.634971e+05,-8.110793e+05,-3.800732e+06,-5.849294e+03,-450619.831440,-1.327957e+06,-3.875384e+06,-2.276433e+04,-1.495841e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,"Retail trade, except of motor vehicles and mot...",-1.010736e+05,-7.059217e+05,-1.100655e+05,-3.292632e+06,-2.184806e+06,-5.071892e+05,-1.516787e+06,-7.884424e+05,-1.724398e+06,-222065.999277,-5.627341e+05,-1.586013e+07,-8.513858e+04,-2.766139e+07
9009,Manufacture of basic iron and steel and of fer...,-1.429076e+05,-1.272571e+06,-1.426185e+05,-2.290660e+07,-2.562011e+06,-2.845541e+06,-6.021346e+07,-6.562955e+07,-3.528015e+07,-335318.520693,-1.596656e+07,-2.102340e+08,-1.083696e+06,-4.186150e+08
9010,Quarrying of sand and clay,-6.184726e+04,-3.444812e+06,-2.346714e+06,-3.869935e+05,-7.091088e+05,-2.617897e+04,-9.948010e+04,-1.104070e+07,-1.024349e+05,-72067.302974,-3.921155e+05,-2.190611e+05,-1.214363e+06,-2.011587e+07
9011,Quarrying of sand and clay,-1.377242e+05,-1.148233e+06,-1.504722e+05,-2.910874e+06,-1.210038e+05,-4.785289e+06,-1.625373e+06,-3.047654e+07,-6.031294e+06,-153659.199910,-8.957421e+05,-3.301965e+08,-8.637923e+04,-3.787191e+08


## Feature engineering

In [13]:
def _remap_industry(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    industries_to_keep = df['Industry (Exiobase)'].value_counts()[df['Industry (Exiobase)'].value_counts() > 50].index
    df['Industry (Exiobase)'] = df['Industry (Exiobase)'].apply(lambda x: x if x in industries_to_keep else 'Other')
    return df

In [14]:
def _create_interaction_terms(df: pd.DataFrame) -> pd.DataFrame:
    interaction_pairs = [
        ('Use of Sold Products', 'Processing of Sold Products'),
        ('Use of Sold Products', 'Purchased Goods and Services'),
        ('Processing of Sold Products', 'Purchased Goods and Services'),
        ('Purchased Goods and Services', 'End of Life Treatment of Sold Products')
    ]
    
    for col1, col2 in interaction_pairs:
        new_col_name = f"{col1}_x_{col2}"
        df[new_col_name] = df[col1] * df[col2]
        
    return df

In [15]:
def _create_polynomial_features(df: pd.DataFrame) -> pd.DataFrame:
    cols_to_square = [
        'Use of Sold Products', 
        'Processing of Sold Products', 
        'Purchased Goods and Services', 
        'End of Life Treatment of Sold Products'
    ]
    
    for col in cols_to_square:
        new_col_name = f"{col}_Squared"
        df[new_col_name] = df[col] ** 2
    
    return df


## Encoding Categorical Variables

In [16]:
def _one_hot_encode(df: pd.DataFrame) -> pd.DataFrame:
    # One-hot encode 'Country' and 'Industry (Exiobase)' columns
    df_encoded = pd.get_dummies(df, columns=['Industry (Exiobase)'])
    return df_encoded

## Normalization/Standardization 

In [17]:
def _normalization(df: pd.DataFrame) -> pd.DataFrame:
    # Create the scaler
    scaler_standard = StandardScaler()

    # Fit the scaler to the data (excluding categorical data if not already encoded)
    df_normalized_standard = pd.DataFrame(scaler_standard.fit_transform(df), columns=df.columns)
    
    return df_normalized_standard

In [18]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """
    Conducts feature engineering on the given DataFrame.

    Steps:
    1. Outlier Removal: Removes outliers using the Isolation Forest algorithm.
    2. Remap Industry: Aggregates less frequent industry categories into 'Other'.
    3. Create Interaction Terms: Creates new features by multiplying pairs of existing features.
    4. Create Polynomial Features: Squares selected features to create new polynomial features.
    5. One-Hot Encoding: One-hot encodes categorical features.
    6. Normalization: Standardizes the feature values.

    Args:
        df: Original DataFrame.

    Returns:
        df_feature_engineered: DataFrame after feature engineering.
    """

    df = _outlier_removal(df)
    df = _remap_industry(df)
    df = _create_interaction_terms(df)
    df = _create_polynomial_features(df)
    df = _one_hot_encode(df)
    df = _normalization(df)
    df_feature_engineered = df

    return df_feature_engineered

In [19]:
df_feature_engineered = feature_engineering(preprocessed_df)

In [20]:
df_feature_engineered

,Business Travel,Capital Goods,Downstream Leased Assets,Downstream Transportation and Distribution,Employee Commuting,End of Life Treatment of Sold Products,Fuel-and-energy-related activities (not included in Scope 1 or 2),Processing of Sold Products,Purchased Goods and Services,Upstream Leased Assets,...,Industry (Exiobase)_Production of electricity nec,Industry (Exiobase)_Public administration and defence; compulsory social security (75),"Industry (Exiobase)_Publishing, printing and reproduction of recorded media (22)",Industry (Exiobase)_Quarrying of sand and clay,Industry (Exiobase)_Real estate activities (70),Industry (Exiobase)_Renting of machinery and equipment without operator and of personal and household goods (71),Industry (Exiobase)_Research and development (73),"Industry (Exiobase)_Retail trade, except of motor vehicles and motorcycles; repair of personal and household goods (52)",Industry (Exiobase)_Sea and coastal water transport,Industry (Exiobase)_Transport via railways
0,0.422912,0.661295,0.303409,0.443951,0.501955,0.321028,0.262314,0.289646,0.440976,0.383294,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
1,0.398137,0.662173,0.290040,0.414812,0.510812,0.321690,0.262283,0.347704,0.440726,0.370265,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
2,0.406296,-2.572262,0.238169,0.426699,0.444091,0.321690,0.231288,0.194535,0.361686,0.311113,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
3,-0.128509,0.489232,0.262660,0.348408,0.371699,0.198620,0.236858,0.232739,0.410029,0.385130,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
4,0.282708,0.517253,0.288853,0.414847,0.223178,0.315444,0.249709,0.289646,0.441125,0.157887,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7205,0.481757,0.615107,0.290040,0.263238,-0.117066,0.304374,0.238603,0.347704,0.433701,0.311786,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,4.407372,-0.086054,-0.093888
7206,0.465189,0.575352,0.284287,-0.823149,-0.227623,0.198154,-0.685098,-0.902024,0.288739,0.235526,...,-0.105254,-0.110517,-0.147243,-0.149198,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
7207,0.497292,0.422953,-0.105281,0.424177,0.315453,0.326224,0.260907,0.150105,0.440708,0.412790,...,-0.105254,-0.110517,-0.147243,6.702505,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888
7208,0.467242,0.584076,0.282899,0.284383,0.487824,0.110041,0.236894,-0.224495,0.415095,0.357849,...,-0.105254,-0.110517,-0.147243,6.702505,-0.174053,-0.105925,-0.121566,-0.226893,-0.086054,-0.093888


In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [22]:
parameters = catalog.load("params:model_options")

[10/09/23 13:39:33] INFO     Loading data from 'params:model_options' (MemoryDataset)...        ]8;id=739145;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=461121;file:///home/adam_przychodni/miniconda3/envs/kedro-environment/lib/python3.10/site-packages/kedro/io/data_catalog.py#492\492]8;;\

In [23]:
def split_data(data: pd.DataFrame, model_options: Dict) -> Tuple:
    """Splits data into features and targets training and test sets.

    Args:
        data: Data containing features and target.
        parameters: Parameters defined in parameters/data_science.yml.
    Returns:
        Split data.
    """
    # X = data[parameters["features"]]
    X = data[parameters["features"]].drop("Scope 3", axis=1)
    y = data["Scope 3"]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=parameters["test_size"], random_state=parameters["random_state"]
    )
    return X_train, X_test, y_train, y_test

In [24]:
# Assuming df_feature_engineered is your DataFrame and parameters is your configuration dictionary
X_train, X_test, y_train, y_test = split_data(df_feature_engineered, parameters)

In [28]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import logging
import pandas as pd
from typing import Any

def train_model(X_train: pd.DataFrame, y_train: pd.Series) -> Any:
    """Trains the XGBoost model.
    
    Args:
        X_train: Training data of independent features.
        y_train: Training data for target variable.
        
    Returns:
        Trained model.
    """
    params = {
        'alpha': 9.418025790529975e-05,
        'colsample_bytree': 0.73850137825373,
        'eta': 0.03756810920990241,
        'gamma': 1.8103086083962833e-05,
        'lambda': 0.006052853661670603,
        'max_depth': 4,
        'min_child_weight': 1.0000000000000004e-06,
        'objective': 'reg:squarederror',
        'subsample': 0.8954379516782436,
        'eval_metric': ['rmse', 'mae']
    }
    dtrain = xgb.DMatrix(X_train, label=y_train)
    model = xgb.train(params, dtrain, num_boost_round=674)
    return model

In [ ]:
def evaluate_model(model: Any, X_test: pd.DataFrame, y_test: pd.Series):
    """Calculates and logs the coefficient of determination and RMSE.
    
    Args:
        model: Trained XGBoost model.
        X_test: Testing data of independent features.
        y_test: Testing data for target variable.
    """
    dtest = xgb.DMatrix(X_test, label=y_test)
    y_pred = model.predict(dtest)
    score = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    print(f"Model has a coefficient R^2 of {score:.3f} on test data.")
    print(f"Model has a RMSE of {rmse:.3f} on test data.")
    
    logger = logging.getLogger(__name__)
    logger.info(f"Model has a coefficient R^2 of {score:.3f} on test data.")
    logger.info(f"Model has a RMSE of {rmse:.3f} on test data.")


In [29]:
# Train the model
trained_model = train_model(X_train, y_train)

In [30]:
# Evaluate the model
evaluate_model(trained_model, X_test, y_test)

Model has a coefficient R^2 of 0.988 on test data.
Model has a RMSE of 0.119 on test data.
